<a href="https://colab.research.google.com/github/chunbo777/dacon_prac/blob/main/%EC%98%81%ED%99%94%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [73]:
!pip install chart_studio

In [74]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from chart_studio import plotly
from plotly import *
import plotly.graph_objects as go

In [75]:
train = pd.read_csv("/content/drive/MyDrive/dacon_practice/영화 관객수/movies_train.csv")
test = ("/content/drive/MyDrive/dacon_practice/영화 관객수/movies_test.csv")

In [76]:
train.head()

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num
0,개들의 전쟁,롯데엔터테인먼트,액션,2012-11-22,96,청소년 관람불가,조병옥,NaN,0,91,2,23398
1,내부자들,(주)쇼박스,느와르,2015-11-19,130,청소년 관람불가,우민호,1161602.50,2,387,3,7072501
2,은밀하게 위대하게,(주)쇼박스,액션,2013-06-05,123,15세 관람가,장철수,220775.25,4,343,4,6959083
3,나는 공무원이다,(주)NEW,코미디,2012-07-12,101,전체 관람가,구자홍,23894.00,2,20,6,217866
4,불량남녀,쇼박스(주)미디어플렉스,코미디,2010-11-04,108,15세 관람가,신근호,1.00,1,251,2,483387


"""
movies_train.csv / movies_test.csv

title : 영화의 제목

distributor : 배급사

genre : 장르

release_time : 개봉일

time : 상영시간(분)

screening_rat : 상영등급

director : 감독이름

dir_prev_bfnum : 해당 감독이 이 영화를 만들기 전 제작에 참여한 영화에서의 평균 관객수(단 관객수가 알려지지 않은 영화 제외)

dir_prev_num : 해당 감독이 이 영화를 만들기 전 제작에 참여한 영화의 개수(단 관객수가 알려지지 않은 영화 제외)

num_staff : 스텝수

num_actor : 주연배우수

box_off_num : 관객수
"""

In [77]:
train.isna().sum()

title               0
distributor         0
genre               0
release_time        0
time                0
screening_rat       0
director            0
dir_prev_bfnum    330
dir_prev_num        0
num_staff           0
num_actor           0
box_off_num         0
dtype: int64

In [78]:
train.describe()

,time,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num
count,600.000000,2.700000e+02,600.000000,600.000000,600.000000,6.000000e+02
mean,100.863333,1.050443e+06,0.876667,151.118333,3.706667,7.081818e+05
std,18.097528,1.791408e+06,1.183409,165.654671,2.446889,1.828006e+06
min,45.000000,1.000000e+00,0.000000,0.000000,0.000000,1.000000e+00
25%,89.000000,2.038000e+04,0.000000,17.000000,2.000000,1.297250e+03
50%,100.000000,4.784236e+05,0.000000,82.500000,3.000000,1.259100e+04
75%,114.000000,1.286569e+06,2.000000,264.000000,4.000000,4.798868e+05
max,180.000000,1.761531e+07,5.000000,869.000000,25.000000,1.426277e+07


이전 관객수 컬럼의 반 이상이 결측치임.

In [79]:
train["date"] = pd.DatetimeIndex(train["release_time"], yearfirst =True)

In [80]:
train.set_index(train["date"], inplace =True)

In [81]:
train.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 600 entries, 2012-11-22 to 2013-01-30
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   title           600 non-null    object        
 1   distributor     600 non-null    object        
 2   genre           600 non-null    object        
 3   release_time    600 non-null    object        
 4   time            600 non-null    int64         
 5   screening_rat   600 non-null    object        
 6   director        600 non-null    object        
 7   dir_prev_bfnum  270 non-null    float64       
 8   dir_prev_num    600 non-null    int64         
 9   num_staff       600 non-null    int64         
 10  num_actor       600 non-null    int64         
 11  box_off_num     600 non-null    int64         
 12  date            600 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(5), object(6)
memory usage: 65.6+ KB


# 개봉일이 영향을 주나?

In [82]:
train["month"] = train.index.month
train["year"] = train.index.year
train["weekday"] = train.index.weekday

In [89]:
train.head()

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num,date,month,year,weekday
date,,,,,,,,,,,,,,,,
2012-11-22,개들의 전쟁,롯데엔터테인먼트,액션,2012-11-22,96,청소년 관람불가,조병옥,NaN,0,91,2,23398,2012-11-22,11,2012,3
2015-11-19,내부자들,(주)쇼박스,느와르,2015-11-19,130,청소년 관람불가,우민호,1161602.50,2,387,3,7072501,2015-11-19,11,2015,3
2013-06-05,은밀하게 위대하게,(주)쇼박스,액션,2013-06-05,123,15세 관람가,장철수,220775.25,4,343,4,6959083,2013-06-05,6,2013,2
2012-07-12,나는 공무원이다,(주)NEW,코미디,2012-07-12,101,전체 관람가,구자홍,23894.00,2,20,6,217866,2012-07-12,7,2012,3
2010-11-04,불량남녀,쇼박스(주)미디어플렉스,코미디,2010-11-04,108,15세 관람가,신근호,1.00,1,251,2,483387,2010-11-04,11,2010,3


In [92]:
train["day"] = pd.DatetimeIndex(train["release_time"], yearfirst =True)

In [93]:
daily_num = train.groupby(["day"], as_index = False)["box_off_num"].sum()

In [94]:
monthly_num = train.groupby(["day", "month"], as_index = False)["box_off_num"].sum()
yearly_num = train.groupby(["day", "year"], as_index = False)["box_off_num"].sum()

In [95]:
monthly_num

,day,month,box_off_num
0,2010-01-02,1,353
1,2010-01-14,1,326613
2,2010-01-21,1,730977
3,2010-01-28,1,462714
4,2010-02-04,2,2821
...,...,...,...
325,2015-12-16,12,9522444
326,2015-12-17,12,1599
327,2015-12-24,12,329
328,2015-12-30,12,627586


In [97]:
data = go.Scatter(x = daily_num["day"],
           y = daily_num["box_off_num"])
layout = go.Layout(title = "일자에 따른 관객 수",
                   xaxis = dict(title = "일자"),
                   yaxis = dict(title = "관객수"))
fig = go.Figure(data = data,
                layout = layout)
fig.show(renderer = "colab")

In [104]:
import plotly.express as px
items = []
# for i, m in enumerate(monthly_num["month"].unique()):
#     current_items = monthly_num[(monthly_num["month"] == m)]
#     items.append(go.Bar(x = current_items["day"],
#                             y = current_items["box_off_num"],
#                             name = ("%s월" %m)))

# layout = go.Layout(title = "월에 따른 관객 수",
#                    xaxis = dict(title = "월"),
#                    yaxis = dict(title = "관객수"))
# fig = go.Figure(data = items,
#                 layout = layout)
# fig.show(renderer = "colab")


fig = px.bar(monthly_num, x="month", y="box_off_num")
fig.show()

In [105]:

fig = px.scatter(yearly_num, x="day", y="box_off_num", color="year"
                 ,size='box_off_num')
fig.show()

성수기와와 비성수기가가 존재. 햇수가 올라갈수록 약간 증가 추세

#감독들의 필모가 영향 주나?

In [106]:
train.head()

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num,date,month,year,weekday,day
date,,,,,,,,,,,,,,,,,
2012-11-22,개들의 전쟁,롯데엔터테인먼트,액션,2012-11-22,96,청소년 관람불가,조병옥,NaN,0,91,2,23398,2012-11-22,11,2012,3,2012-11-22
2015-11-19,내부자들,(주)쇼박스,느와르,2015-11-19,130,청소년 관람불가,우민호,1161602.50,2,387,3,7072501,2015-11-19,11,2015,3,2015-11-19
2013-06-05,은밀하게 위대하게,(주)쇼박스,액션,2013-06-05,123,15세 관람가,장철수,220775.25,4,343,4,6959083,2013-06-05,6,2013,2,2013-06-05
2012-07-12,나는 공무원이다,(주)NEW,코미디,2012-07-12,101,전체 관람가,구자홍,23894.00,2,20,6,217866,2012-07-12,7,2012,3,2012-07-12
2010-11-04,불량남녀,쇼박스(주)미디어플렉스,코미디,2010-11-04,108,15세 관람가,신근호,1.00,1,251,2,483387,2010-11-04,11,2010,3,2010-11-04


In [107]:
fig = px.scatter(train, x="dir_prev_num", y="box_off_num")
fig.show()

In [110]:
fig = px.scatter(train, x="num_actor", y="box_off_num")
fig.show()

In [71]:
fig = px.scatter(train, x="dir_prev_bfnum", y="box_off_num")
fig.show()

In [111]:
fig = px.scatter(train, x="time", y="box_off_num")
fig.show()

In [117]:
fig = px.scatter(train, x="num_staff", y="box_off_num")
fig.show()

In [114]:
fig = px.bar(train, x="distributor", y="box_off_num")
fig.show()

In [115]:
fig = px.bar(train, x="genre", y="box_off_num")
fig.show()

영향력이 큰 순= 배급사(상위10개) > 이전필모 > 개봉월 > 개봉년>

In [116]:
train.describe()

,time,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num,month,year,weekday
count,600.000000,2.700000e+02,600.000000,600.000000,600.000000,6.000000e+02,600.000000,600.000000,600.000000
mean,100.863333,1.050443e+06,0.876667,151.118333,3.706667,7.081818e+05,7.011667,2012.748333,2.801667
std,18.097528,1.791408e+06,1.183409,165.654671,2.446889,1.828006e+06,3.394712,1.727224,0.471913
min,45.000000,1.000000e+00,0.000000,0.000000,0.000000,1.000000e+00,1.000000,2010.000000,0.000000
25%,89.000000,2.038000e+04,0.000000,17.000000,2.000000,1.297250e+03,4.000000,2011.000000,3.000000
50%,100.000000,4.784236e+05,0.000000,82.500000,3.000000,1.259100e+04,7.000000,2013.000000,3.000000
75%,114.000000,1.286569e+06,2.000000,264.000000,4.000000,4.798868e+05,10.000000,2014.000000,3.000000
max,180.000000,1.761531e+07,5.000000,869.000000,25.000000,1.426277e+07,12.000000,2015.000000,5.000000


In [119]:
train_filled = train.fillna(train.mean())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning:

DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.



In [123]:
train.duplicated().sum()

0

In [ ]:
for dis in train["distributor"].unique():
    if disD[dis]
        disD[dis]+=1

In [138]:
grouped_dis = train.groupby("distributor")["title"].count()
core_distributors = grouped_dis.sort_values(ascending=False)[:20]

In [144]:
train_filled["distributor"][3]

'(주)NEW'

In [145]:
for i, t in enumerate(train_filled["distributor"]):
    if t not in list(core_distributors.index):
        train_filled["distributor"][i] = "기타"


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [147]:
train_filled.head()

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num,date,month,year,weekday,day
date,,,,,,,,,,,,,,,,,
2012-11-22,개들의 전쟁,롯데엔터테인먼트,액션,2012-11-22,96,청소년 관람불가,조병옥,1.050443e+06,0,91,2,23398,2012-11-22,11,2012,3,2012-11-22
2015-11-19,내부자들,(주)쇼박스,느와르,2015-11-19,130,청소년 관람불가,우민호,1.161602e+06,2,387,3,7072501,2015-11-19,11,2015,3,2015-11-19
2013-06-05,은밀하게 위대하게,(주)쇼박스,액션,2013-06-05,123,15세 관람가,장철수,2.207752e+05,4,343,4,6959083,2013-06-05,6,2013,2,2013-06-05
2012-07-12,나는 공무원이다,(주)NEW,코미디,2012-07-12,101,전체 관람가,구자홍,2.389400e+04,2,20,6,217866,2012-07-12,7,2012,3,2012-07-12
2010-11-04,불량남녀,기타,코미디,2010-11-04,108,15세 관람가,신근호,1.000000e+00,1,251,2,483387,2010-11-04,11,2010,3,2010-11-04
